In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/LT-EDI@LDK 2025/Caste and Migration Hate Speech Detection/train.csv')
augmented_dataset = pd.read_csv('/content/drive/MyDrive/LT-EDI@LDK 2025/Caste and Migration Hate Speech Detection/augmented_dataset.csv')
test = pd.read_csv('/content/drive/MyDrive/LT-EDI@LDK 2025/Caste and Migration Hate Speech Detection/dev.csv')
rt = pd.read_csv('/content/drive/MyDrive/LT-EDI@LDK 2025/Caste and Migration Hate Speech Detection/test.csv')

In [ ]:
rt

,id,text
0,4594,Humanity paatha intha slag countries prblm var...
1,3593,Avanga Holi kondada oruku poranga da
2,2533,Avangala avanga ooruke anupura vazhiya parunga.
3,6975,பைத்தியமாடா நீ.. நான் சோழிய வேளாளர். நான் என் ...
4,6011,Caste is there parayan think to marry higheru...
...,...,...
1571,6360,Ivlo varalaru pesarelada kena...aprm ethuku da...
1572,6208,@dineshkumar6695 தம்பி. யார் பேச்சையும் கேட...
1573,6871,பார்ப்பானும் பறையனும் ஒன்னு பழமொழி உள்ளது 😁😂
1574,442,நம்ம இடத்தை காப்பாத்திக்கிரது பேரு ரவுடிசம்னா ...


In [ ]:
train

,id,text,label
0,3268,Indha ariya kandupidippin moolam neenga solla ...,0
1,6239,@vijayakumarp7959 unmai therincha nee pesu,0
2,5859,Inga erukka yella dev... boys vadakkan vadakk...,1
3,3519,பீகாரி பிரசாந்த் கிஷோரிடம் கொடுத்த 350 கோடியை ...,1
4,5136,Mumbai Bangalore la 80% percentage outsiders,1
...,...,...,...
5507,3273,வாழ்க வளமுடன் வணக்கம் எப்படி இருக்கீங்க எல்லார...,0
5508,156,இந்தியா ஓட மொழி ஹிந்தி னு சொனகா தமிழ்நாடு மட்ட...,1
5509,1911,நாம் தமிழர் என்ற தேசிய இனத்தின் பெருமைமிக்க இன...,0
5510,6267,you build temple in your own cost \nand see if...,0


In [ ]:
train["label"].value_counts()

,count
label,
0,3415
1,2097


In [ ]:
augmented_dataset["label"].value_counts()


,count
label,
0,3415
1,3347


In [ ]:
test["label"].value_counts()

,count
label,
0,485
1,302


# Preprocessing

In [ ]:
!pip install indic-nlp-library pandas nltk scikit-learn



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.2 MB/s eta 0:00:00


In [ ]:

import pandas as pd
import re
from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory


# Load dataset
df_train = train
df_test = test
df_aug = augmented_dataset

# Tamil Normalizer
factory = IndicNormalizerFactory()
normalizer = factory.get_normalizer("ta")  # Tamil Normalizer

# Tamil Stopwords (Custom List)
tamil_stopwords = set([
    "மற்றும்", "என்பது", "இந்த", "அதனால்", "இது", "அவர்கள்", "உள்ளது", "இல்லை", "ஆகவே", "அவர்", "என்ற", "உள்ளன"
])

def clean_tamil_mixed_text(text):
    """Preprocess Tamil + English mixed text: normalize, tokenize, keep English words."""
    text = str(text).strip()  # Convert to string and remove extra spaces
    text = normalizer.normalize(text)  # Normalize Tamil text

    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)

    # Remove special characters except Tamil & English
    text = re.sub(r"[^஀-௿a-zA-Z\s]", "", text)

    # Tokenize (Handles Tamil + English words)
    words = indic_tokenize.trivial_tokenize(text)

    # Remove stopwords
    words = [word for word in words if word.lower() not in tamil_stopwords]

    return " ".join(words)

# Apply preprocessing
df_train["clean_text"] = df_train["text"].astype(str).apply(clean_tamil_mixed_text)
df_test["clean_text"] = df_test["text"].astype(str).apply(clean_tamil_mixed_text)
df_aug["clean_text"] = df_aug["clean_text"].astype(str).apply(clean_tamil_mixed_text)


# # Handle Class Imbalance using Oversampling (Optional)
# class_0 = df_test[df_test["label"] == 0]
# class_1 = df_test[df_test["label"] == 1]

# if len(class_0) > len(class_1):  # Oversample minority class
#     class_1_upsampled = resample(class_1, replace=True, n_samples=len(class_0), random_state=42)
#     df_balanced = pd.concat([class_0, class_1_upsampled])
# else:
#     class_0_upsampled = resample(class_0, replace=True, n_samples=len(class_1), random_state=42)
#     df_balanced = pd.concat([class_0_upsampled, class_1])

# Shuffle dataset
# df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# # Save processed dataset
# df_balanced.to_csv("processed_tamil_dataset.csv", index=False)

# # Check results
# print(df_balanced.head())


In [ ]:
rt["clean_text"] = rt["text"].astype(str).apply(clean_tamil_mixed_text)

In [ ]:
rt["clean_text"]

,clean_text
0,Humanity paatha intha slag countries prblm var...
1,Avanga Holi kondada oruku poranga da
2,Avangala avanga ooruke anupura vazhiya parunga
3,பைத்தியமாடா நீ நான் சோழிய வேளாளர் நான் என் அப்...
4,Caste is there parayan think to marry higherup...
...,...
1571,Ivlo varalaru pesarelada kenaaprm ethuku da pa...
1572,dineshkumar தம்பி யார் பேச்சையும் கேட்காதே கடந...
1573,பார்ப்பானும் பறையனும் ஒன்னு பழமொழி
1574,நம்ம இடத்தை காப்பாத்திக்கிரது பேரு ரவுடிசம்னா ...


In [ ]:
df_train

,id,text,label,clean_text
0,3268,Indha ariya kandupidippin moolam neenga solla ...,0,Indha ariya kandupidippin moolam neenga solla ...
1,6239,@vijayakumarp7959 unmai therincha nee pesu,0,vijayakumarp unmai therincha nee pesu
2,5859,Inga erukka yella dev... boys vadakkan vadakk...,1,Inga erukka yella dev boys vadakkan vadakkan s...
3,3519,பீகாரி பிரசாந்த் கிஷோரிடம் கொடுத்த 350 கோடியை ...,1,பீகாரி பிரசாந்த் கிஷோரிடம் கொடுத்த கோடியை எப்ப...
4,5136,Mumbai Bangalore la 80% percentage outsiders,1,Mumbai Bangalore la percentage outsiders
...,...,...,...,...
5507,3273,வாழ்க வளமுடன் வணக்கம் எப்படி இருக்கீங்க எல்லார...,0,வாழ்க வளமுடன் வணக்கம் எப்படி இருக்கீங்க எல்லார...
5508,156,இந்தியா ஓட மொழி ஹிந்தி னு சொனகா தமிழ்நாடு மட்ட...,1,இந்தியா ஓட மொழி ஹிந்தி னு சொனகா தமிழ்நாடு மட்ட...
5509,1911,நாம் தமிழர் என்ற தேசிய இனத்தின் பெருமைமிக்க இன...,0,நாம் தமிழர் தேசிய இனத்தின் பெருமைமிக்க இன மக்க...
5510,6267,you build temple in your own cost \nand see if...,0,you build temple in your own cost \nand see if...


#Data augmentation

In [ ]:
!pip install indic-transliteration googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=6858156e1337e15dd5b061bf01a3c832c486140e5ab446fe568e990dbd1a4725
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperf

In [ ]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
import googletrans

translator = googletrans.Translator(raise_exception=True)

def back_translate_tamil(text):
    en_text = translator.translate(text, src='ta', dest='en').text
    ta_text = translator.translate(en_text, src='en', dest='ta').text
    return ta_text

text = "நா இன்று மகிழ்ச்சி உணர்கிறேன்"
augmented_text = back_translate_tamil(text)
print(augmented_text)



நா இன்று மகிழ்ச்சியாக உணர்கிறார்


In [ ]:
import random
import numpy as np
import pandas as pd
import googletrans
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
from sentence_transformers import SentenceTransformer, util

sbert_model = SentenceTransformer("paraphrase-mpnet-base-v2")

df_label_1 = df[df["label"] == 1]
df_label_0 = df[df["label"] == 0]

samples_needed = len(df_label_0) - len(df_label_1)

df_sampled = df_label_1.sample(n=samples_needed, random_state=42)

translator = googletrans.Translator()

def back_translate_tamil(text):
    """Perform Tamil → English → Tamil back-translation."""
    try:
        en_text = translator.translate(text, src="ta", dest="en").text
        ta_text = translator.translate(en_text, src="en", dest="ta").text
        return ta_text
    except Exception as e:
        return text

df_sampled["augmented_text"] = df_sampled["clean_text"].apply(back_translate_tamil)

def is_similar(original, augmented, threshold=0.5):
    """Check cosine similarity between original and augmented sentences."""
    original_embedding = sbert_model.encode(original, convert_to_tensor=True)
    augmented_embedding = sbert_model.encode(augmented, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(original_embedding, augmented_embedding).item()
    return similarity >= threshold
df_sampled["similar"] = df_sampled.apply(lambda row: is_similar(row["clean_text"], row["augmented_text"]), axis=1)

df_augmented = df_sampled[df_sampled["similar"]][["augmented_text"]].rename(columns={"augmented_text": "clean_text"})
df_augmented["label"] = 1
df_balanced = pd.concat([df, df_augmented], ignore_index=True)

print(df_balanced["label"].value_counts())

label
0    3415
1    3347
Name: count, dtype: int64


In [ ]:
df_balanced.to_csv("/content/drive/MyDrive/LT-EDI@LDK 2025/Caste and Migration Hate Speech Detection/augmented_dataset.csv", index=False)

In [ ]:
df_balanced["label"].value_counts()

NameError: name 'df_balanced' is not defined

# Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(df_train["clean_text"], df_train["label"], test_size=0.2, random_state=42)

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))  # Unigram + Bigram
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a Logistic Regression Model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate Model Performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 0.71
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.88      0.80       722
           1       0.62      0.38      0.47       381

    accuracy                           0.71      1103
   macro avg       0.67      0.63      0.63      1103
weighted avg       0.69      0.71      0.68      1103



In [ ]:

X_new_test_tfidf = vectorizer.transform(df_test["clean_text"])

y_new_pred = model.predict(X_new_test_tfidf)

if "label" in df_test.columns:
    accuracy = accuracy_score(df_test["label"], y_new_pred)
    print(f"New Test Data Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(df_test["label"], y_new_pred))
else:
    print("Predictions on New Test Data:", y_new_pred)


New Test Data Accuracy: 0.71
Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.90      0.79       485
           1       0.72      0.39      0.51       302

    accuracy                           0.71       787
   macro avg       0.71      0.65      0.65       787
weighted avg       0.71      0.71      0.68       787



## Logistic Regression With balanced data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(df_aug["clean_text"], df_aug["label"], test_size=0.2, random_state=42)

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf_balanced = vectorizer.fit_transform(X_train_balanced)
X_test_tfidf_balanced = vectorizer.transform(X_test_balanced)

# Train a Logistic Regression Model
model_balanced = LogisticRegression()
model_balanced.fit(X_train_tfidf_balanced, y_train_balanced)

# Predictions
y_pred_balanced = model_balanced.predict(X_test_tfidf_balanced)

# Evaluate Model Performance
accuracy = accuracy_score(y_test_balanced, y_pred_balanced)
print(f"Model Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test_balanced, y_pred_balanced))

Model Accuracy: 0.61
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.39      0.50       680
           1       0.57      0.83      0.68       673

    accuracy                           0.61      1353
   macro avg       0.63      0.61      0.59      1353
weighted avg       0.63      0.61      0.59      1353



In [ ]:

X_new_test_tfidf = vectorizer.transform(df_test["clean_text"])

y_new_pred = model_balanced.predict(X_new_test_tfidf)

if "label" in df_test.columns:
    accuracy = accuracy_score(df_test["label"], y_new_pred)
    print(f"New Test Data Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(df_test["label"], y_new_pred))
else:
    print("Predictions on New Test Data:", y_new_pred)


New Test Data Accuracy: 0.55
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.41      0.53       485
           1       0.45      0.78      0.57       302

    accuracy                           0.55       787
   macro avg       0.60      0.60      0.55       787
weighted avg       0.64      0.55      0.55       787



# XG Boost

In [ ]:
import xgboost as xgb

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=8000, ngram_range=(1,2))  # Unigram + Bigram
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train XGBoost Model
xgb_model = xgb.XGBClassifier(objective="binary:logistic", n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
xgb_model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = xgb_model.predict(X_test_tfidf)

# Model Performance
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Model Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

XGBoost Model Accuracy: 0.70
Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.89      0.80       722
           1       0.62      0.35      0.45       381

    accuracy                           0.70      1103
   macro avg       0.67      0.62      0.62      1103
weighted avg       0.69      0.70      0.68      1103



In [ ]:
X_new_test_tfidf = vectorizer.transform(df_test["clean_text"])

# Predict labels for new test data
y_new_pred = xgb_model.predict(X_new_test_tfidf)

# If your test dataset has actual labels, evaluate accuracy
if "label" in df_test.columns:
    accuracy = accuracy_score(df_test["label"], y_new_pred)
    print(f"New Test Data Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(df_test["label"], y_new_pred))
else:
    print("Predictions on New Test Data:", y_new_pred)

New Test Data Accuracy: 0.71
Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.90      0.79       485
           1       0.71      0.40      0.51       302

    accuracy                           0.71       787
   macro avg       0.71      0.65      0.65       787
weighted avg       0.71      0.71      0.68       787



## XG Boosting with balanced data

In [ ]:
import xgboost as xgb

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=8000, ngram_range=(1,2))  # Unigram + Bigram
X_train_tfidf_balanced = vectorizer.fit_transform(X_train_balanced)
X_test_tfidf_balanced = vectorizer.transform(X_test_balanced)

# Train XGBoost Model
xgb_model_balanced = xgb.XGBClassifier(objective="binary:logistic", n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
xgb_model_balanced.fit(X_train_tfidf_balanced, y_train_balanced)

# Predictions
y_pred_balanced = xgb_model_balanced.predict(X_test_tfidf_balanced)

# Evaluate Model Performance
accuracy = accuracy_score(y_test_balanced, y_pred_balanced)
print(f"XGBoost Model Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test_balanced, y_pred_balanced))

NameError: name 'X_train_balanced' is not defined

In [ ]:
X_new_test_tfidf = vectorizer.transform(df_test["clean_text"])

y_new_pred = xgb_model_balanced.predict(X_new_test_tfidf)

if "label" in df_test.columns:
    accuracy = accuracy_score(df_test["label"], y_new_pred)
    print(f"New Test Data Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(df_test["label"], y_new_pred))
else:
    print("Predictions on New Test Data:", y_new_pred)

New Test Data Accuracy: 0.55
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.38      0.51       485
           1       0.45      0.83      0.59       302

    accuracy                           0.55       787
   macro avg       0.62      0.60      0.55       787
weighted avg       0.66      0.55      0.54       787



#SVM

In [ ]:
from sklearn.svm import SVC

# Train SVM Model
svm_model = SVC(kernel="linear", C=1.0)
svm_model.fit(X_train_tfidf, y_train)

# Predictions
y_pred_svm = svm_model.predict(X_test_tfidf)

# Evaluate Model Performance
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Model Accuracy: {accuracy_svm:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred_svm))

SVM Model Accuracy: 0.71
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.83      0.79       722
           1       0.60      0.48      0.53       381

    accuracy                           0.71      1103
   macro avg       0.67      0.65      0.66      1103
weighted avg       0.70      0.71      0.70      1103



In [ ]:
X_new_test_tfidf = vectorizer.transform(df_test["clean_text"])

y_new_pred = svm_model.predict(X_new_test_tfidf)

if "label" in df_test.columns:
    accuracy = accuracy_score(df_test["label"], y_new_pred)
    print(f"New Test Data Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(df_test["label"], y_new_pred))
else:
    print("Predictions on New Test Data:", y_new_pred)


New Test Data Accuracy: 0.73
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.85      0.80       485
           1       0.69      0.54      0.61       302

    accuracy                           0.73       787
   macro avg       0.72      0.70      0.70       787
weighted avg       0.73      0.73      0.72       787



## SVM with balanced data

In [ ]:
from sklearn.svm import SVC

# Train SVM Model
svm_model_balanced = SVC(kernel="linear", C=1.0)
svm_model_balanced.fit(X_train_tfidf_balanced, y_train_balanced)

# Predictions
y_pred_svm_balanced = svm_model_balanced.predict(X_test_tfidf_balanced)

# Evaluate Model Performance
accuracy_svm = accuracy_score(y_test_balanced, y_pred_svm_balanced)
print(f"SVM Model Accuracy: {accuracy_svm:.2f}")
print("Classification Report:\n", classification_report(y_test_balanced, y_pred_svm_balanced))

SVM Model Accuracy: 0.62
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.39      0.51       680
           1       0.58      0.85      0.69       673

    accuracy                           0.62      1353
   macro avg       0.65      0.62      0.60      1353
weighted avg       0.65      0.62      0.60      1353



In [ ]:
X_new_test_tfidf = vectorizer.transform(df_test["clean_text"])

y_new_pred = svm_model_balanced.predict(X_new_test_tfidf)

if "label" in df_test.columns:
    accuracy = accuracy_score(df_test["label"], y_new_pred)
    print(f"New Test Data Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(df_test["label"], y_new_pred))
else:
    print("Predictions on New Test Data:", y_new_pred)


New Test Data Accuracy: 0.56
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.40      0.53       485
           1       0.46      0.82      0.59       302

    accuracy                           0.56       787
   macro avg       0.62      0.61      0.56       787
weighted avg       0.66      0.56      0.55       787



# Stacking classifier (xgb+svm)

In [ ]:
from sklearn.ensemble import StackingClassifier

# TF-IDF transformation
vectorizer = TfidfVectorizer(max_features=8000, ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Define base models
base_models = [
    ('xgb', xgb.XGBClassifier(objective="binary:logistic", n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)),
    ('svm', SVC(kernel="linear", C=1.0, probability=True))
]

# Meta learner (final classifier)
meta_model = LogisticRegression()

# Stacking classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Train model
stacking_model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = stacking_model.predict(X_test_tfidf)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Stacking Hybrid Model Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Stacking Hybrid Model Accuracy: 0.72
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.82      0.79       722
           1       0.61      0.52      0.56       381

    accuracy                           0.72      1103
   macro avg       0.68      0.67      0.68      1103
weighted avg       0.71      0.72      0.71      1103



In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Convert test data to TF-IDF format using the same vectorizer
X_test_new = vectorizer.transform(df_test["clean_text"])

# True labels
y_test_new = df_test["label"]

# Make predictions using the trained model
y_pred_new = stacking_model.predict(X_test_new)

# classification report
print("\n🔹 Classification Report:")
print(classification_report(y_test_new, y_pred_new))

# accuracy
accuracy = accuracy_score(y_test_new, y_pred_new)
print(f"\n✅ Accuracy on New Test Dataset: {accuracy:.2f}")

# confusion matrix
conf_matrix = confusion_matrix(y_test_new, y_pred_new)
print("\n🌀 Confusion Matrix:\n", conf_matrix)


🔹 Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.84      0.80       485
           1       0.70      0.59      0.64       302

    accuracy                           0.74       787
   macro avg       0.73      0.72      0.72       787
weighted avg       0.74      0.74      0.74       787


✅ Accuracy on New Test Dataset: 0.74

🌀 Confusion Matrix:
 [[407  78]
 [123 179]]


In [ ]:
# Transform `rt` dataset using the same TF-IDF vectorizer
X_rt_tfidf = vectorizer.transform(rt["clean_text"])

# Predict using the trained voting model
rt["predictions"] = stacking_model.predict(X_rt_tfidf)

# Print or save the predictions
print(rt.head())  # Show some results
rt.to_csv("/content/drive/MyDrive/LT-EDI@LDK 2025/Caste and Migration Hate Speech Detection/rt_voting_predictions_3.csv", index=False)  # Save predictions if needed


     id                                               text  \
0  4594  Humanity paatha intha slag countries prblm var...   
1  3593               Avanga Holi kondada oruku poranga da   
2  2533    Avangala avanga ooruke anupura vazhiya parunga.   
3  6975  பைத்தியமாடா நீ.. நான் சோழிய வேளாளர். நான் என் ...   
4  6011  Caste is there  parayan think to marry higheru...   

                                          clean_text  predictions  
0  Humanity paatha intha slag countries prblm var...            1  
1               Avanga Holi kondada oruku poranga da            0  
2     Avangala avanga ooruke anupura vazhiya parunga            1  
3  பைத்தியமாடா நீ நான் சோழிய வேளாளர் நான் என் அப்...            0  
4  Caste is there parayan think to marry higherup...            0  


In [ ]:
rt["predictions"].value_counts()

,count
predictions,
0,1098
1,478


## Stacking classifier (xgb+svm) with balanced data

In [ ]:
from sklearn.ensemble import StackingClassifier

# TF-IDF transformation
vectorizer = TfidfVectorizer(max_features=8000, ngram_range=(1,2))
X_train_tfidf_balanced = vectorizer.fit_transform(X_train_balanced)
X_test_tfidf_balanced = vectorizer.transform(X_test_balanced)

# Define base models
base_models_balanced = [
    ('xgb', xgb.XGBClassifier(objective="binary:logistic", n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)),
    ('svm', SVC(kernel="linear", C=1.0, probability=True))
]

# Meta learner (final classifier)
meta_model_balanced = LogisticRegression()

# Stacking classifier
stacking_model_balanced = StackingClassifier(estimators=base_models_balanced, final_estimator=meta_model_balanced)

# Train model
stacking_model_balanced.fit(X_train_tfidf_balanced, y_train_balanced)

# Predictions
y_pred = stacking_model.predict(X_test_tfidf_balanced)

# Evaluation
accuracy = accuracy_score(y_test_balanced, y_pred)
print(f"Stacking Hybrid Model Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test_balanced, y_pred))

Stacking Hybrid Model Accuracy: 0.50
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.99      0.67       680
           1       0.56      0.01      0.03       673

    accuracy                           0.50      1353
   macro avg       0.53      0.50      0.35      1353
weighted avg       0.53      0.50      0.35      1353



In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Convert test data to TF-IDF format using the same vectorizer
X_test_new = vectorizer.transform(df_test["clean_text"])

# True labels
y_test_new = df_test["label"]

# Make predictions using the trained model
y_pred_new = stacking_model_balanced.predict(X_test_new)

# classification report
print("\n🔹 Classification Report:")
print(classification_report(y_test_new, y_pred_new))

# accuracy
accuracy = accuracy_score(y_test_new, y_pred_new)
print(f"\n✅ Accuracy on New Test Dataset: {accuracy:.2f}")

# confusion matrix
conf_matrix = confusion_matrix(y_test_new, y_pred_new)
print("\n🌀 Confusion Matrix:\n", conf_matrix)


🔹 Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.42      0.55       485
           1       0.47      0.81      0.59       302

    accuracy                           0.57       787
   macro avg       0.63      0.62      0.57       787
weighted avg       0.66      0.57      0.57       787


✅ Accuracy on New Test Dataset: 0.57

🌀 Confusion Matrix:
 [[204 281]
 [ 56 246]]


# Voting classifier (xgb+svm+nb)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB

# Define base models
xgb_clf = xgb.XGBClassifier(objective="binary:logistic", n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
svm_clf = SVC(kernel="linear", C=1.0, probability=True)
nb_clf = MultinomialNB()

# Voting classifier (hard voting)
voting_model = VotingClassifier(estimators=[
    ('xgb', xgb_clf), ('svm', svm_clf), ('nb', nb_clf)
], voting='hard')

# Train model
voting_model.fit(X_train_tfidf, y_train)

# Predictions
y_pred_voting = voting_model.predict(X_test_tfidf)

# Evaluation
accuracy_voting = accuracy_score(y_test, y_pred_voting)
print(f"Voting Hybrid Model Accuracy: {accuracy_voting:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred_voting))

Voting Hybrid Model Accuracy: 0.73
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.91      0.81       722
           1       0.68      0.38      0.49       381

    accuracy                           0.73      1103
   macro avg       0.71      0.64      0.65      1103
weighted avg       0.72      0.73      0.70      1103



In [ ]:


# Convert test data to TF-IDF format using the same vectorizer
X_test_new = vectorizer.transform(df_test["clean_text"])

# True labels (if available in test set)
y_test_new = df_test["label"]

# Make predictions using the trained voting model
y_pred_voting_new = voting_model.predict(X_test_new)

# Generate classification report
print("\n🔹 Classification Report for Voting Classifier:")
print(classification_report(y_test_new, y_pred_voting_new))

# Compute accuracy
accuracy_voting_new = accuracy_score(y_test_new, y_pred_voting_new)
print(f"\n✅ Accuracy on New Test Dataset: {accuracy_voting_new:.2f}")

# confusion matrix
conf_matrix_voting = confusion_matrix(y_test_new, y_pred_voting_new)
print("\n🌀 Confusion Matrix:\n", conf_matrix_voting)



🔹 Classification Report for Voting Classifier:
              precision    recall  f1-score   support

           0       0.72      0.90      0.80       485
           1       0.73      0.43      0.54       302

    accuracy                           0.72       787
   macro avg       0.72      0.67      0.67       787
weighted avg       0.72      0.72      0.70       787


✅ Accuracy on New Test Dataset: 0.72

🌀 Confusion Matrix:
 [[437  48]
 [172 130]]


In [ ]:
# Transform `rt` dataset using the same TF-IDF vectorizer
X_rt_tfidf = vectorizer.transform(rt["clean_text"])

# Predict using the trained voting model
rt["predictions"] = voting_model.predict(X_rt_tfidf)

# Print or save the predictions
print(rt.head())  # Show some results
rt.to_csv("/content/drive/MyDrive/LT-EDI@LDK 2025/Caste and Migration Hate Speech Detection/rt_voting_predictions_2.csv", index=False)  # Save predictions if needed


     id                                               text  \
0  4594  Humanity paatha intha slag countries prblm var...   
1  3593               Avanga Holi kondada oruku poranga da   
2  2533    Avangala avanga ooruke anupura vazhiya parunga.   
3  6975  பைத்தியமாடா நீ.. நான் சோழிய வேளாளர். நான் என் ...   
4  6011  Caste is there  parayan think to marry higheru...   

                                          clean_text  predictions  
0  Humanity paatha intha slag countries prblm var...            0  
1               Avanga Holi kondada oruku poranga da            0  
2     Avangala avanga ooruke anupura vazhiya parunga            1  
3  பைத்தியமாடா நீ நான் சோழிய வேளாளர் நான் என் அப்...            0  
4  Caste is there parayan think to marry higherup...            0  


In [ ]:
rt['predictions'].value_counts()

,count
predictions,
0,1250
1,326


## Voting classifier (xgb+svm+nb) with balanced data

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB

# Define base models
xgb_clf_balanced = xgb.XGBClassifier(objective="binary:logistic", n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
svm_clf_balanced = SVC(kernel="linear", C=1.0, probability=True)
nb_clf_balanced = MultinomialNB()

# Voting classifier (hard voting)
voting_model_balanced = VotingClassifier(estimators=[
    ('xgb', xgb_clf_balanced), ('svm', svm_clf_balanced), ('nb', nb_clf_balanced)
], voting='hard')

# Train model
voting_model_balanced.fit(X_train_tfidf_balanced, y_train_balanced)

# Predictions
y_pred_voting = voting_model_balanced.predict(X_test_tfidf_balanced)

# Evaluation
accuracy_voting = accuracy_score(y_test_balanced, y_pred_voting)
print(f"Voting Hybrid Model Accuracy: {accuracy_voting:.2f}")
print("Classification Report:\n", classification_report(y_test_balanced, y_pred_voting))

Voting Hybrid Model Accuracy: 0.61
Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.39      0.50       680
           1       0.57      0.84      0.68       673

    accuracy                           0.61      1353
   macro avg       0.64      0.61      0.59      1353
weighted avg       0.64      0.61      0.59      1353



In [ ]:


# Convert test data to TF-IDF format using the same vectorizer
X_test_new = vectorizer.transform(df_test["clean_text"])

# True labels
y_test_new = df_test["label"]

# predictions using the trained voting model
y_pred_voting_new = voting_model_balanced.predict(X_test_new)

# classification report
print("\n Classification Report for Voting Classifier:")
print(classification_report(y_test_new, y_pred_voting_new))

# accuracy
accuracy_voting_new = accuracy_score(y_test_new, y_pred_voting_new)
print(f"\n Accuracy on New Test Dataset: {accuracy_voting_new:.2f}")

# confusion matrix
conf_matrix_voting = confusion_matrix(y_test_new, y_pred_voting_new)
print("\n Confusion Matrix:\n", conf_matrix_voting)



 Classification Report for Voting Classifier:
              precision    recall  f1-score   support

           0       0.77      0.41      0.53       485
           1       0.46      0.80      0.58       302

    accuracy                           0.56       787
   macro avg       0.61      0.60      0.56       787
weighted avg       0.65      0.56      0.55       787


 Accuracy on New Test Dataset: 0.56

 Confusion Matrix:
 [[197 288]
 [ 60 242]]


#  BiLSTM + Attention Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Input, AdditiveAttention
from sklearn.metrics import classification_report
# Global Pooling to reduce sequence dimension
from tensorflow.keras.layers import GlobalMaxPooling1D

# Parameters
MAX_VOCAB = 10000  # Max unique words
MAX_LENGTH = 100   # Max words per text
EMBEDDING_DIM = 128

# Tokenization
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(df_train["clean_text"])

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(df_train["clean_text"])
X_test_seq = tokenizer.texts_to_sequences(df_test["clean_text"])

# Pad sequences
X_train_seq = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding="post", truncating="post")
X_test_seq = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding="post", truncating="post")

# Labels
y_train = df_train["label"].values
y_test = df_test["label"].values
# Input Layer
inputs = Input(shape=(MAX_LENGTH,))

# Embedding Layer
embedding = Embedding(input_dim=MAX_VOCAB, output_dim=128)(inputs)

# BiLSTM Layer
bilstm = Bidirectional(LSTM(128, return_sequences=True))(embedding)  # Outputs 256 dim


pooled = GlobalMaxPooling1D()(bilstm)

# Attention Mechanism (optional, can be simplified)
attention = Dense(256, activation="tanh")(pooled)

# Fully Connected Layers
dense = Dense(64, activation='relu')(attention)
dropout = Dropout(0.4)(dense)
outputs = Dense(1, activation='sigmoid')(dropout)  # Binary Classification

# Model
model = Model(inputs=inputs, outputs=outputs)

# Compile Model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train_seq, y_train, epochs=10, batch_size=32, validation_data=(X_test_seq, y_test))

# Evaluate Model
y_pred_dl = (model.predict(X_test_seq) > 0.5).astype("int32")
print("\nClassification Report:\n", classification_report(y_test, y_pred_dl))


Epoch 1/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.6120 - loss: 0.6640 - val_accuracy: 0.6607 - val_loss: 0.6088
Epoch 2/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.8415 - loss: 0.3705 - val_accuracy: 0.7408 - val_loss: 0.5284
Epoch 3/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9396 - loss: 0.1598 - val_accuracy: 0.7687 - val_loss: 0.5644
Epoch 4/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9783 - loss: 0.0758 - val_accuracy: 0.7598 - val_loss: 0.7729
Epoch 5/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9871 - loss: 0.0393 - val_accuracy: 0.7776 - val_loss: 0.8653
Epoch 6/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9871 - loss: 0.0316 - val_accuracy: 0.8018 - val_loss: 1.1395
Epoch 7/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9915 - loss: 0.0287 - val_accuracy: 0.7891 - val_loss: 1.2841
Epoch 8/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9901 - loss: 0.0240 - val_accu

In [ ]:
# Tokenize and pad the new dataset `rt`
X_rt_seq = tokenizer.texts_to_sequences(rt["clean_text"])
X_rt_seq = pad_sequences(X_rt_seq, maxlen=MAX_LENGTH, padding="post", truncating="post")

# Predict using the trained model
rt["predictions"] = (model.predict(X_rt_seq) > 0.5).astype("int32")

# Print or save the predictions
print(rt.head())  # Show some results
rt.to_csv("/content/drive/MyDrive/LT-EDI@LDK 2025/Caste and Migration Hate Speech Detection/rt_predictions_1.csv", index=False)


50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
     id                                               text  \
0  4594  Humanity paatha intha slag countries prblm var...   
1  3593               Avanga Holi kondada oruku poranga da   
2  2533    Avangala avanga ooruke anupura vazhiya parunga.   
3  6975  பைத்தியமாடா நீ.. நான் சோழிய வேளாளர். நான் என் ...   
4  6011  Caste is there  parayan think to marry higheru...   

                                          clean_text  predictions  
0  Humanity paatha intha slag countries prblm var...            1  
1               Avanga Holi kondada oruku poranga da            0  
2     Avangala avanga ooruke anupura vazhiya parunga            0  
3  பைத்தியமாடா நீ நான் சோழிய வேளாளர் நான் என் அப்...            0  
4  Caste is there parayan think to marry higherup...            0  


In [ ]:
rt["predictions"].value_counts()

,count
predictions,
0,982
1,594


## BiLSTM + Attention Model with balanced data

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Input, AdditiveAttention
from sklearn.metrics import classification_report
# Global Pooling to reduce sequence dimension
from tensorflow.keras.layers import GlobalMaxPooling1D

# Parameters
MAX_VOCAB = 10000  # Max unique words
MAX_LENGTH = 100   # Max words per text
EMBEDDING_DIM = 128

# Tokenization
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(df_aug["clean_text"])

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(df_aug["clean_text"])
X_test_seq = tokenizer.texts_to_sequences(df_test["clean_text"])

# Pad sequences
X_train_seq = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding="post", truncating="post")
X_test_seq = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding="post", truncating="post")

# Labels
y_train = df_aug["label"].values
y_test = df_test["label"].values
# Input Layer
inputs = Input(shape=(MAX_LENGTH,))

# Embedding Layer
embedding = Embedding(input_dim=MAX_VOCAB, output_dim=128)(inputs)

# BiLSTM Layer
bilstm = Bidirectional(LSTM(128, return_sequences=True))(embedding)  # Outputs 256 dim


pooled = GlobalMaxPooling1D()(bilstm)

# Attention Mechanism (optional, can be simplified)
attention = Dense(256, activation="tanh")(pooled)

# Fully Connected Layers
dense = Dense(64, activation='relu')(attention)
dropout = Dropout(0.4)(dense)
outputs = Dense(1, activation='sigmoid')(dropout)  # Binary Classification

# Model
model = Model(inputs=inputs, outputs=outputs)

# Compile Model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train_seq, y_train, epochs=10, batch_size=32, validation_data=(X_test_seq, y_test))

# Evaluate Model
y_pred_dl = (model.predict(X_test_seq) > 0.5).astype("int32")
print("\nClassification Report:\n", classification_report(y_test, y_pred_dl))


Epoch 1/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.5255 - loss: 0.6897 - val_accuracy: 0.6531 - val_loss: 0.6363
Epoch 2/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.6671 - loss: 0.6186 - val_accuracy: 0.5845 - val_loss: 0.6077
Epoch 3/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7201 - loss: 0.4101 - val_accuracy: 0.5680 - val_loss: 0.6596
Epoch 4/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7525 - loss: 0.3711 - val_accuracy: 0.5820 - val_loss: 0.7728
Epoch 5/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7437 - loss: 0.3578 - val_accuracy: 0.5743 - val_loss: 0.9929
Epoch 6/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7471 - loss: 0.3540 - val_accuracy: 0.5934 - val_loss: 1.1129
Epoch 7/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.7515 - loss: 0.3512 - val_accuracy: 0.5997 - val_loss: 0.8197
Epoch 8/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7487 - loss: 0.3575 - val_accu

# xlm-roberta-base

In [ ]:
!pip install transformers tensorflow scikit-learn numpy

In [ ]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.metrics import classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight

# Load Dataset
X_train = df_train["clean_text"].tolist()
y_train = df_train["label"].values

X_test = df_test["clean_text"].tolist()
y_test = df_test["label"].values

# Load XLM-RoBERTa Model and Tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize Dataset
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128, return_tensors="tf")
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128, return_tensors="tf")

# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': train_encodings['input_ids'],
        'attention_mask': train_encodings['attention_mask']
    },
    y_train
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': test_encodings['input_ids'],
        'attention_mask': test_encodings['attention_mask']
    },
    y_test
)).batch(16)

# Compute Class Weights for Handling Imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}


# Define Weighted Loss Function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile Model with Accuracy
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Train Model with Class Weights
model.fit(train_dataset, epochs=5, class_weight=class_weights_dict)

# Function to Evaluate Model on a Dataset
def evaluate_model(dataset):
    y_preds = []
    y_true = []

    for batch_inputs, batch_labels in dataset:
        logits = model(batch_inputs, training=False)['logits']
        batch_preds = tf.argmax(logits, axis=-1)

        y_preds.extend(batch_preds.numpy())
        y_true.extend(batch_labels.numpy())

    return y_true, y_preds

# Evaluate on Train Set
y_train_true, y_train_preds = evaluate_model(train_dataset)
print("\nTrain Set Classification Report:")
print(classification_report(y_train_true, y_train_preds))

# Evaluate on Test Set
y_test_true, y_test_preds = evaluate_model(test_dataset)
print("\nTest Set Classification Report:")
print(classification_report(y_test_true, y_test_preds))

# Calculate F1 Scores for Train Set
macro_f1_train = f1_score(y_train_true, y_train_preds, average='macro')
weighted_f1_train = f1_score(y_train_true, y_train_preds, average='weighted')
print("\nTrain Macro F1 Score:", macro_f1_train)
print("Train Weighted F1 Score:", weighted_f1_train)

# Calculate F1 Scores for Test Set
macro_f1_test = f1_score(y_test_true, y_test_preds, average='macro')
weighted_f1_test = f1_score(y_test_true, y_test_preds, average='weighted')
print("\nTest Macro F1 Score:", macro_f1_test)
print("Test Weighted F1 Score:", weighted_f1_test)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
345/345 [==============================] - 197s 446ms/step - loss: 0.6894 - accuracy: 0.5945
Epoch 2/5
345/345 [==============================] - 148s 428ms/step - loss: 0.6774 - accuracy: 0.6081
Epoch 3/5
345/345 [==============================] - 148s 428ms/step - loss: 0.6743 - accuracy: 0.6154
Epoch 4/5
345/345 [==============================] - 147s 427ms/step - loss: 0.6787 - accuracy: 0.6123
Epoch 5/5
345/345 [==============================] - 148s 427ms/step - loss: 0.6722 - accuracy: 0.6156

Train Set Classification Report:
              precision    recall  f1-score   support

           0       0.62      1.00      0.77      3415
           1       0.00      0.00      0.00      2097

    accuracy                           0.62      5512
   macro avg       0.31      0.50      0.38      5512
weighted avg       0.38      0.62      0.47      5512



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.62      1.00      0.76       485
           1       0.00      0.00      0.00       302

    accuracy                           0.62       787
   macro avg       0.31      0.50      0.38       787
weighted avg       0.38      0.62      0.47       787


Train Macro F1 Score: 0.38254732832978605
Train Weighted F1 Score: 0.4740200022664076

Test Macro F1 Score: 0.3812893081761006
Test Weighted F1 Score: 0.46994997322848486


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## xlm-roberta-base with balanced data

In [ ]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.metrics import classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight

# Load Dataset
X_train = df_aug["clean_text"].tolist()
y_train = df_aug["label"].values

X_test = df_test["clean_text"].tolist()
y_test = df_test["label"].values

# Load XLM-RoBERTa Model and Tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize Dataset
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128, return_tensors="tf")
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128, return_tensors="tf")

# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': train_encodings['input_ids'],
        'attention_mask': train_encodings['attention_mask']
    },
    y_train
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': test_encodings['input_ids'],
        'attention_mask': test_encodings['attention_mask']
    },
    y_test
)).batch(16)

# Compute Class Weights for Handling Imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}


# Define Weighted Loss Function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile Model with Accuracy
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Train Model with Class Weights
model.fit(train_dataset, epochs=5, class_weight=class_weights_dict)

# Function to Evaluate Model on a Dataset
def evaluate_model(dataset):
    y_preds = []
    y_true = []

    for batch_inputs, batch_labels in dataset:
        logits = model(batch_inputs, training=False)['logits']
        batch_preds = tf.argmax(logits, axis=-1)

        y_preds.extend(batch_preds.numpy())
        y_true.extend(batch_labels.numpy())

    return y_true, y_preds

# Evaluate on Train Set
y_train_true, y_train_preds = evaluate_model(train_dataset)
print("\nTrain Set Classification Report:")
print(classification_report(y_train_true, y_train_preds))

# Evaluate on Test Set
y_test_true, y_test_preds = evaluate_model(test_dataset)
print("\nTest Set Classification Report:")
print(classification_report(y_test_true, y_test_preds))

# Calculate F1 Scores for Train Set
macro_f1_train = f1_score(y_train_true, y_train_preds, average='macro')
weighted_f1_train = f1_score(y_train_true, y_train_preds, average='weighted')
print("\nTrain Macro F1 Score:", macro_f1_train)
print("Train Weighted F1 Score:", weighted_f1_train)

# Calculate F1 Scores for Test Set
macro_f1_test = f1_score(y_test_true, y_test_preds, average='macro')
weighted_f1_test = f1_score(y_test_true, y_test_preds, average='weighted')
print("\nTest Macro F1 Score:", macro_f1_test)
print("Test Weighted F1 Score:", weighted_f1_test)


All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
423/423 [==============================] - 240s 440ms/step - loss: 0.5652 - accuracy: 0.6637
Epoch 2/5
423/423 [==============================] - 180s 426ms/step - loss: 0.6543 - accuracy: 0.6384
Epoch 3/5
423/423 [==============================] - 181s 427ms/step - loss: 0.7133 - accuracy: 0.6473
Epoch 4/5
423/423 [==============================] - 182s 431ms/step - loss: 0.7248 - accuracy: 0.6420
Epoch 5/5
423/423 [==============================] - 181s 427ms/step - loss: 0.7175 - accuracy: 0.6390

Train Set Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3415
           1       0.49      1.00      0.66      3347

    accuracy                           0.49      6762
   macro avg       0.25      0.50      0.33      6762
weighted avg       0.24      0.49      0.33      6762



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       485
           1       0.38      1.00      0.55       302

    accuracy                           0.38       787
   macro avg       0.19      0.50      0.28       787
weighted avg       0.15      0.38      0.21       787


Train Macro F1 Score: 0.3310911069344149
Train Weighted F1 Score: 0.32776158973957015

Test Macro F1 Score: 0.2773186409550046
Test Weighted F1 Score: 0.21283412850930467


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
